# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
establishments_collection = db['establishments']
one_document = establishments_collection.find_one()
pprint(one_document)

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65efd1e97821949ae3b72ff4'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [9]:
# Insert the new restaurant into the collection
result = establishments_collection.insert_one(new_restaurant_data)

In [10]:
# Check that the new restaurant was inserted
print("Inserted new restaurant with ID:", result.inserted_id)

Inserted new restaurant with ID: 65efd24c9983fc07889d8ca4


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
restaurant_cafe_canteen = establishments_collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1})

pprint(restaurant_cafe_canteen)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to find the new restaurant
filter_query = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the BusinessTypeID
update_operation = {"$set": {"BusinessTypeID": restaurant_cafe_canteen["BusinessTypeID"]}}

# Update the new restaurant with the correct BusinessTypeID
update_result = establishments_collection.update_one(filter_query, update_operation)

In [13]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Restaurant updated successfully.")
else:
    print("No matching restaurant found.")

Restaurant updated successfully.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})
print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)

Number of documents with LocalAuthorityName as 'Dover': 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many({"LocalAuthorityName": "Dover"})
print("Number of documents deleted:", delete_result.deleted_count)

Number of documents deleted: 994


In [16]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})

if remaining_dover_count == 0:
    print("No remaining documents include Dover in the LocalAuthorityName.")
else:
    print("There are still remaining documents with Dover in the LocalAuthorityName:", remaining_dover_count)

No remaining documents include Dover in the LocalAuthorityName.


In [17]:
# Check that other documents remain with 'find_one'
remaining_document = establishments_collection.find_one()

if remaining_document is not None:
    print("At least one document remains in the collection.")
    pprint(remaining_document)
else:
    print("No documents found in the collection.")

At least one document remains in the collection.
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65efd1e97821949ae3b732db'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 's

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments_collection.update_many({}, [
    {
        "$set": {
            "geocode.latitude": {"$toDouble": "$geocode.latitude"},
            "geocode.longitude": {"$toDouble": "$geocode.longitude"}
        }
    }
])

print("Number of documents updated:", update_result.modified_count)

Number of documents updated: 38786


Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
update_result = establishments_collection.update_many({}, [
    {
        "$set": {
            "RatingValue": {"$toInt": "$RatingValue"}
        }
    }
])

print("Number of documents updated:", update_result.modified_count)

Number of documents updated: 34695


In [21]:
# Check that the coordinates and rating value are now numbers
one_document = establishments_collection.find_one()

print("Data type of longitude:", type(one_document["geocode"]["longitude"]))
print("Data type of latitude:", type(one_document["geocode"]["latitude"]))
print("Data type of RatingValue:", type(one_document["RatingValue"]))

Data type of longitude: <class 'float'>
Data type of latitude: <class 'float'>
Data type of RatingValue: <class 'int'>
